# Project 1: Digit Classification with KNN and Naive Bayes

In this project, you'll implement your own image recognition system for classifying digits. Read through the code and the instructions carefully and add your own code where indicated. Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but <b> please prepare your own write-up (with your own code). </b>

If you're interested, check out these links related to digit recognition:

Yann Lecun's MNIST benchmarks: http://yann.lecun.com/exdb/mnist/

Stanford Streetview research and data: http://ufldl.stanford.edu/housenumbers/

In [ ]:
## NOTE: this project was done with sklearn 0.20.0

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_mldata
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

Load the data. Notice that we are splitting the data into training, development, and test. We also have a small subset of the training data called mini_train_data and mini_train_labels that you should use in all the experiments below, unless otherwise noted.

In [ ]:
# Load the digit data either from mldata.org, or once downloaded to data_home, from disk. The data is about 53MB so this cell
# should take a while the first time your run it.
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target

# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

(1) Create a 10x10 grid to visualize 10 examples of each digit. Python hints:

- plt.rc() for setting the colormap, for example to black and white
- plt.subplot() for creating subplots
- plt.imshow() for rendering a matrix
- np.array.reshape() for reshaping a 1D feature vector into a 2D matrix (for rendering)

In [ ]:
#def P1(num_examples=10):

### STUDENT START ###
def create_examples_dict(train_labels, num_examples):
    examples_dict = {num: [] for num in range(10)}
    sets_found = set()
    for i, label in enumerate(train_labels):
        if len(sets_found) == num_examples:
            break
        if len(examples_dict[label]) < num_examples:
            examples_dict[label].append(i)
        else:
            sets_found.add(label)
    return examples_dict

def plot_examples(examples_dict, train_data, num_examples):
    # loop over subplots and access correct matrix
    # for each value in each dictionary entry
    f, axarr = plt.subplots(len(examples_dict),num_examples, figsize=(15,15))
    for i in range(len(examples_dict)):
        for j in range(num_examples):
            desired_matrix = mini_train_data[examples_dict[i][j]]
            im_to_show = np.reshape(desired_matrix, (28, 28))
            axarr[i,j].imshow(im_to_show, cmap=plt.cm.Greys)
            axarr[i,j].axis('off')

number_of_examples = 10
ex_dict = create_examples_dict(mini_train_labels, number_of_examples)
plot_examples(ex_dict, mini_train_data, number_of_examples)
### STUDENT END ###

#P1(10)

(2) Evaluate a K-Nearest-Neighbors model with k = [1,3,5,7,9] using the mini training set. Report accuracy on the dev set. For k=1, show precision, recall, and F1 for each label. Which is the most difficult digit?

- KNeighborsClassifier() for fitting and predicting
- classification_report() for producing precision, recall, F1 results

In [ ]:
#def P2(k_values):

### STUDENT START ###
k_values = [1, 3, 5, 7, 9]

def train_k_neighbors(k_values):
    for k in k_values:
        k_neighbors = KNeighborsClassifier(k)
        k_neighbors.fit(mini_train_data, mini_train_labels)
        dev_predictions = k_neighbors.predict(dev_data)
        print("Accuracy when k={}: ".format(k), np.sum(dev_predictions==dev_labels) / 1000)
        if k==1:
            print("\nClassification report when k=1:")
            print(classification_report(dev_labels, dev_predictions))

train_k_neighbors(k_values)    
### STUDENT END ###

#k_values = [1, 3, 5, 7, 9]
#P2(k_values)

ANSWER: The most difficult digit for precision is 3, which means that of all the digits the model predicts to be 3's, a smaller percentage of those predictions are correct compared to the other labels. The most difficult digit for recall is 2, which means that the model incorrectly predicts on more 2's than it does on other labels. Overall, the digit with the lowest f1 score (combination of precision and recall) is 9.

(3) Using k=1, report dev set accuracy for the training set sizes below. Also, measure the amount of time needed for prediction with each training size.

- time.time() gives a wall clock value you can use for timing operations

In [ ]:
#def P3(train_sizes, accuracies):

### STUDENT START ###

train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25000]
accuracies = []

def train_k_neighbors_by_size(train_sizes, accuracies):
    for size in train_sizes:
        k_neighbors = KNeighborsClassifier(1)
        k_neighbors.fit(train_data[:size], train_labels[:size])
        start_time = time.time()
        dev_predictions = k_neighbors.predict(dev_data)
        end_time = time.time()
        accuracy = np.sum(dev_predictions==dev_labels) / 1000
        accuracies.append(accuracy)
        prediction_time = end_time - start_time
        print("Training set size: {}".format(size))
        print("Accuracy: {}".format(accuracy))
        print("Time taken for 1000 predictions: {:.2f} seconds\n".format(prediction_time))

train_k_neighbors_by_size(train_sizes, accuracies)
### TODO: use model.score for accuracy 

### STUDENT END ###

#train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25000]
#accuracies = []
#P3(train_sizes, accuracies)

(4) Fit a regression model that predicts accuracy from training size. What does it predict for n=60000? What's wrong with using regression here? Can you apply a transformation that makes the predictions more reasonable?

- Remember that the sklearn fit() functions take an input matrix X and output vector Y. So each input example in X is a vector, even if it contains only a single value.

In [ ]:
#def P4():

### STUDENT START ###

# turn train sizes list into array and reshape into 2-d matrix
def fit_accuracy_predictor(train_sizes_array, 
                           accuracies, 
                           value_to_predict,
                           message):
    # create and fit linear regression model
    lr = LinearRegression()
    lr.fit(train_sizes_array, accuracies)

    print("Prediction for training size 60,000 with {}: {:.2f}".format(message,
                                                                       lr.predict(value_to_predict)[0]))

train_sizes_array = np.asarray(train_sizes)
train_sizes_array = np.reshape(train_sizes_array, (len(train_sizes_array), 1))

sixty_thousand = np.array([60000]).reshape(-1,1)

# original linear model fit
fit_accuracy_predictor(train_sizes_array, 
                       accuracies, 
                       sixty_thousand, 
                       "no transformation")

# trying again with a log transform
fit_accuracy_predictor(np.log(train_sizes_array),
                      accuracies,
                      np.log(sixty_thousand),
                      "log transform of training size")

### STUDENT END ###

#P4()

ANSWER: When we regress accuracy on training examples, the model predicts that a training size of 60,000 will lead to an accuracy of 1.243, which is not possible, since the most highest accuracy a model can have is 1 (100% accurate). We need to apply a transformation to the output variable such that the accuracy values will be modeled as approaching 1. A good way of doing this is applying a log transformation to the training size. This effectively models the way that accuracy gains decrease as we continue to up the training size. We still get a prediction that is slightly over 1, but we can effectively understand this to mean that the regression predicts perfect or near-perfect accuracy at 60,000 training rows

Fit a 1-NN and output a confusion matrix for the dev data. Use the confusion matrix to identify the most confused pair of digits, and display a few example mistakes.

- confusion_matrix() produces a confusion matrix

In [ ]:
#def P5():

### STUDENT START ###

# train, fit, and predict on the classifier
k_neighbors = KNeighborsClassifier(1)
k_neighbors.fit(mini_train_data, mini_train_labels)
dev_predictions = k_neighbors.predict(dev_data)

# show default confusion matrix
cm = confusion_matrix(dev_labels, dev_predictions)
print(cm)

# show prettier confusion matrix
plt.figure(figsize=(8, 8))
plt.title("Confusion Matrix")
plt.ylabel("True Labels")
plt.xlabel("Predicted Labels")
plt.imshow(cm, cmap=plt.cm.Greys)

# find all occurrences where a true 4 is predicted as a 9
true_4s = np.where(dev_labels==4)
predicted_9s = np.where(dev_predictions==9)
true_4s_predicted_9s = np.intersect1d(true_4s, predicted_9s)

# display the examples
f, axarr = plt.subplots(1,5, figsize=(6, 6))
plt.title("True 4s confused for 9s", loc='left')
for i, idx in enumerate(true_4s_predicted_9s[:5]):
    desired_matrix = dev_data[idx]
    im_to_show = np.reshape(desired_matrix, (28, 28))
    axarr[i].imshow(im_to_show)
    axarr[i].axis('off')


### STUDENT END ###

#P5()

(6) A common image processing technique is to smooth an image by blurring. The idea is that the value of a particular pixel is estimated as the weighted combination of the original value and the values around it. Typically, the blurring is Gaussian -- that is, the weight of a pixel's influence is determined by a Gaussian function over the distance to the relevant pixel.

Implement a simplified Gaussian blur by just using the 8 neighboring pixels: the smoothed value of a pixel is a weighted combination of the original value and the 8 neighboring values. Try applying your blur filter in 3 ways:
- preprocess the training data but not the dev data
- preprocess the dev data but not the training data
- preprocess both training and dev data

Note that there are Guassian blur filters available, for example in scipy.ndimage.filters. You're welcome to experiment with those, but you are likely to get the best results with the simplified version I described above.

In [ ]:
#def P6():
    
### STUDENT START ###

def apply_gaussian_blur(orig_matrix):
    orig_matrix = np.reshape(orig_matrix, (28, 28))
    new_matrix = np.copy(orig_matrix)
    
    it = np.nditer(new_matrix, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        # get value for the desired pixel in old matrix
        desired_pixel_orig = orig_matrix[it.multi_index]

        row_idx = it.multi_index[0]
        column_idx = it.multi_index[1]
        
        # get values for the surrounding pixels in old matrix
        surrounding_pixels = []
        
        # loop over the rows
        for i in range(row_idx - 1, row_idx + 2):
            # loop over the columns
            for j in range(column_idx - 1, column_idx + 2):
                # this if statement handles LITERAL edge cases
                # as in, the edges of the image
                if 0 <= i <= 27 and 0 <= j <= 27:
                    if not (i, j) == it.multi_index:
                        surrounding_pixels.append(orig_matrix[i, j])

        # half times original value, half the weight of surrounding values
        new_pixel_value = .5*(desired_pixel_orig) + .5*(np.mean(surrounding_pixels))
        
        # replace it[0] with desired value
        it[0] = new_pixel_value

        it.iternext()
    
        #reshape new matrix to original dimensions
        new_matrix = np.reshape(new_matrix, (784,))
    return new_matrix

# pre-process the train and dev data
blurred_train_data = [apply_gaussian_blur(x) for x in mini_train_data]
blurred_dev_data = [apply_gaussian_blur(x) for x in dev_data]

def train_and_predict(train_option, dev_option, message):
    k_neighbors = KNeighborsClassifier(1)
    k_neighbors.fit(train_option, mini_train_labels)
    dev_predictions = k_neighbors.predict(dev_option)
    accuracy = np.sum(dev_predictions==dev_labels) / 1000
    print("Accuracy with {}: {}".format(message, accuracy))

# compare 1-nn with...

# no blurring
train_and_predict(mini_train_data, dev_data, "no blurring")

# train blurred, dev not blurred
train_and_predict(blurred_train_data, dev_data, "blurring for train, not dev")

# train not blurred, dev blurred
train_and_predict(mini_train_data, blurred_dev_data, "blurring for dev, not train")

# both blurred
train_and_predict(blurred_train_data, blurred_dev_data, "blurring for both train and dev")
### STUDENT END ###

#P6()

ANSWER: The model gets the best accuracy when the training data is blurred, but the dev data isn't. This makes sense to me based on what I know about using dropout in neural networks to prevent overfitting. It helps to train on blurred data, so that you prevent overfitting to the small nuances of your training cases, but it's helpful to keep as much information as possible about the cases you want to predict on. However, the accuracy difference between blurring on dev and not blurring on dev is relatively small, so it seems this could work well either way.

(7) Fit a Naive Bayes classifier and report accuracy on the dev data. Remember that Naive Bayes estimates P(feature|label). While sklearn can handle real-valued features, let's start by mapping the pixel values to either 0 or 1. You can do this as a preprocessing step, or with the binarize argument. With binary-valued features, you can use BernoulliNB. Next try mapping the pixel values to 0, 1, or 2, representing white, grey, or black. This mapping requires MultinomialNB. Does the multi-class version improve the results? Why or why not?

In [ ]:
#def P7():

### STUDENT START ###

bernoulli_nb = BernoulliNB(binarize = 0.33)
bernoulli_nb.fit(mini_train_data, mini_train_labels)
dev_predictions = bernoulli_nb.predict(dev_data)
accuracy = np.sum(dev_predictions==dev_labels) / 1000
print("Accuracy for Bernoulli: {}".format(accuracy))

def preprocess_for_multinomial(image_matrix):
    initial_matrix = (image_matrix > .33).astype(int)
    black_values_matrix = (image_matrix > .66).astype(int)
    final_matrix = initial_matrix + black_values_matrix
    return final_matrix

pre_processed_train_data = []
pre_processed_dev_data = []

for image_matrix in mini_train_data:
    pre_processed_train_data.append(preprocess_for_multinomial(image_matrix))

for image_matrix in dev_data:
    pre_processed_dev_data.append(preprocess_for_multinomial(image_matrix))
    
multinomial_nb = MultinomialNB()
multinomial_nb.fit(pre_processed_train_data, mini_train_labels)
dev_predictions = bernoulli_nb.predict(pre_processed_dev_data)
accuracy = np.sum(dev_predictions==dev_labels) / 1000
print("Accuracy for Multinomial: {}".format(accuracy))

### STUDENT END ###

#P7()

ANSWER: Switching to Multinomial does not improve results. This is most likely because the additional signal created by separating out grey from black pixels isn't really important for determining what label is right for an image. From a number-identifying perspective, what is important is that there is some value in the pixel, indicating the presence of a pen stroke. A certain pixel being more grey than black might have as much to do with the image processing as it does with what the original written digit.

(8) Use GridSearchCV to perform a search over values of alpha (the Laplace smoothing parameter) in a Bernoulli NB model. What is the best value for alpha? What is the accuracy when alpha=0? Is this what you'd expect?

- Note that GridSearchCV partitions the training data so the results will be a bit different than if you used the dev data for evaluation.

In [ ]:
def P8(alphas):

### STUDENT START ###
    nb = BernoulliNB()
    grid_search = GridSearchCV(nb, alphas)
    grid_search.fit(mini_train_data, mini_train_labels)
    return grid_search

alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
nb = P8(alphas)

print(nb.best_params_)
print()
for i in range(len(alphas['alpha'])):  
    print(nb.cv_results_['params'][i])
    print("Accuracy: {}".format(nb.cv_results_['mean_test_score'][i]))
    print()
    
### STUDENT END ###

ANSWER: The best value for alpha is .1, as the classifier has an accuracy of .821. The accuracy when alpha = 0 is .803, which is worse. It makes sense that the value with no smoothing would be worse, because smoothing helps prevent the classifier from overfitting to its training data. We would expect the smoothing effect to be less important as the training size increases. 

(9) Try training a model using GuassianNB, which is intended for real-valued features, and evaluate on the dev data. You'll notice that it doesn't work so well. Try to diagnose the problem. You should be able to find a simple fix that returns the accuracy to around the same rate as BernoulliNB. Explain your solution.

Hint: examine the parameters estimated by the fit() method, theta\_ and sigma\_.

In [ ]:
#def P9():

### STUDENT END ###

def train_gaussian(mini_train_data,
                   mini_train_labels,
                   smoothing):
    gaussian_nb = GaussianNB(var_smoothing = smoothing)
    gaussian_nb.fit(mini_train_data, mini_train_labels)
    return gaussian_nb

def get_accuracy_for_gaussian(model,
                              dev_data,
                              dev_labels,
                              smoothing):
    dev_predictions = model.predict(dev_data)
    accuracy = np.sum(dev_predictions==dev_labels) / 1000
    print("Accuracy for Gaussian with {} value for smoothing: {}".format(smoothing,
                                                                         accuracy))

# the default smoothing value is 1e^-09. this is close
default_smooth = .0001234
gaussian_nb = train_gaussian(mini_train_data,
                            mini_train_labels,
                            default_smooth)
get_accuracy_for_gaussian(gaussian_nb,
                          dev_data,
                          dev_labels,
                          default_smooth)
orig_sigmas = [np.mean(pixel) for pixel in gaussian_nb.sigma_[0]]


# I tested a variety of smooth values and
# this one was best
smooth_value = .08
gaussian_nb_smoothed = train_gaussian(mini_train_data,
                                     mini_train_labels,
                                     smooth_value)
get_accuracy_for_gaussian(gaussian_nb_smoothed,
                          dev_data,
                          dev_labels,
                          smooth_value)

new_sigmas = [np.mean(pixel) for pixel in gaussian_nb_smoothed.sigma_[0]]

bins = np.linspace(0, .3, 30)
plt.hist(orig_sigmas, bins, alpha=0.5, label='Original Sigmas')
plt.hist(new_sigmas, bins, alpha=0.5, label='Smoothed Sigmas')
plt.legend(loc='upper right')
plt.title("Change in Sigma Values for All Pixels in Examples with Label 0")
plt.xlabel("Sigma Value")
plt.ylabel("Number of Pixels")
plt.show()

change_in_sigma = [new_sigmas[i] - orig_sigmas[i] for i in range(len(orig_sigmas))]
print("Average increase in sigma: {}".format(np.mean(change_in_sigma)))
### STUDENT END ###

#gnb = P9()

ANSWER:  A Gaussian model is expecting a normal distribution for the features, whereas for these features, the distribution is not normal at all, and is highly skewed towards 0. As a result, when the model tries to fit the values for each feature to a normal distribution, that distribution ends up being centered very close to 0, with a small variance (small sigmas). Then, when the model sees values for those features that are several standard deviations from the mean, it doesn't know what to do, and guesses randomly. Changing the value of the smoothing parameter causes the model to train such that it finds larger values for sigmas for each feature. With larger variances, the normal distribution for each feature is wider, so more data points fall in the part of the distribution where the model can accurately recognize the value and identify it. The plot shows how much bigger the sigmas are for the pixels in the 0-labeled examples once we add a higher smoothing parameter to the model.

(10) Because Naive Bayes is a generative model, we can use the trained model to generate digits. Train a BernoulliNB model and then generate a 10x20 grid with 20 examples of each digit. Because you're using a Bernoulli model, each pixel output will be either 0 or 1. How do the generated digits compare to the training digits?

- You can use np.random.rand() to generate random numbers from a uniform distribution
- The estimated probability of each pixel is stored in feature\_log\_prob\_. You'll need to use np.exp() to convert a log probability back to a probability.

In [ ]:
#def P10(num_examples):

### STUDENT START ###

def train_bernoulli_nb(binarize):
    bernoulli_nb = BernoulliNB(binarize = binarize)
    bernoulli_nb.fit(mini_train_data, mini_train_labels)
    dev_predictions = bernoulli_nb.predict(dev_data)
    accuracy = np.sum(dev_predictions==dev_labels) / 1000
    print("Accuracy for Bernoulli: {}".format(accuracy))
    return bernoulli_nb

def generate_digits(examples_per_num, nb):
    images_to_show = []
    for item in nb.feature_log_prob_:
        images_per_digit = []
        for i in range(examples_per_num):
            generated_digit = []
            for pixel_prob in np.exp(item):
                pixel_val = 0
                num = np.random.rand()
                if num <= pixel_prob:
                    pixel_val = 1
                generated_digit.append(pixel_val)
            im_to_show = np.reshape(generated_digit, (28, 28))
            images_per_digit.append(im_to_show)
        images_to_show.append(images_per_digit)
    return images_to_show

def plot_generated_digits(images_to_show):
    f, axarr = plt.subplots(10,20, figsize=(15,15))
    for i, examples in enumerate(images_to_show):
        for j, example in enumerate(examples):
            im_to_show = np.reshape(example, (28, 28))
            axarr[i,j].imshow(im_to_show, cmap=plt.cm.Greys)
            axarr[i,j].axis('off')

b_nb = train_bernoulli_nb(.33)
im_to_show = generate_digits(20, b_nb)
plot_generated_digits(im_to_show)

ANSWER: None of the generated images actually look like something someone woud have drawn. That's because each pixel value was determined independently of its neighboring values, based solely on the average value of that pixel over the training examples. With this method, the probabilities ensure that, over 784 pixels, we are likely to activate at least some of the pixels that are typically used for the given number, so we get the overall impression of what number it was. In a real training example, someone draws a digit using a smooth pen stroke, so the likelihood of a certain pixel to be black is very related to the likelihood that its neighbors are black. Since we don't take into account neighboring pixel values in generating these images, they don't mimic the look of a realistic pen stroke.

(11) Remember that a strongly calibrated classifier is rougly 90% accurate when the posterior probability of the predicted class is 0.9. A weakly calibrated classifier is more accurate when the posterior is 90% than when it is 80%. A poorly calibrated classifier has no positive correlation between posterior and accuracy.

Train a BernoulliNB model with a reasonable alpha value. For each posterior bucket (think of a bin in a histogram), you want to estimate the classifier's accuracy. So for each prediction, find the bucket the maximum posterior belongs to and update the "correct" and "total" counters.

How would you characterize the calibration for the Naive Bayes model?

In [ ]:
bernoulli_nb = BernoulliNB(binarize = 0.33, alpha=.1)
bernoulli_nb.fit(mini_train_data, mini_train_labels)
dev_predictions = bernoulli_nb.predict(dev_data)
accuracy = np.sum(dev_predictions==dev_labels) / 1000
print("Accuracy for Bernoulli: {}".format(accuracy))
    
### STUDENT START ###

pred_probs = bernoulli_nb.predict_proba(dev_data)
                
### STUDENT END ###

buckets = [0.5, 0.9, 0.999, 0.99999, 0.9999999, 0.999999999, 0.99999999999, 0.9999999999999, 1.0]

def determine_calibration(prob_predictions, labels, buckets):
    bucket_dict = {b: [] for b in buckets}
    for i, prediction_probs in enumerate(prob_predictions):
        prob = np.amax(prediction_probs)
        pred = np.argmax(prediction_probs)
        accurate_pred_ind = 0
        if pred == labels[i]:
            accurate_pred_ind = 1
        for b in buckets:
            # included predictions at given prob +- .05
            # to get more data points
            # I didn't want to include everything between say, .05 and .09
            # because that bin seems really wide
            if (b - .05) <= prob < (b + .05):
                bucket_dict[b].append(accurate_pred_ind)
                break
    return bucket_dict

final_bucket_dict = determine_calibration(pred_probs,
                                           dev_labels,
                                           buckets)

for bucket, results in final_bucket_dict.items():
    if len(results) > 0:
        accuracy = np.mean(results)
        print("True accuracy for predictions with probability {}: {}".format(bucket, accuracy))
        print("Number of predictions: {}".format(len(results)))

ANSWER: The Naive Bayes classifier appears not to be callibrated at all. While 66.7% of its predictions with probability .5 (+- .05) are in fact, correct, only 45.5% of its predictions with probability .9 (+- .05) are correct, and only 85% of its predictions with probability .999 (+- .035 are correct. If this classifier were well-calibrated, we would see its accuracy go up in lock-step with its prediction probability, and ideally, we'd see its accuracy being very similar to its prediction probability. Here, neither seems to be the case, although we don't see very much data in its smaller buckets to confirm the trend. It appears that the model is typically extremely confident (around 99.9% confident for 94% of its data) in its predictions, even though in reality it is only correct about 83% of the time.

(12) EXTRA CREDIT

Try designing extra features to see if you can improve the performance of Naive Bayes on the dev set. Here are a few ideas to get you started:
- Try summing the pixel values in each row and each column.
- Try counting the number of enclosed regions; 8 usually has 2 enclosed regions, 9 usually has 1, and 7 usually has 0.

Make sure you comment your code well!

In [ ]:
#def P12():

### STUDENT START ###

# trying an ensemble of NB's and K-nns
# and then have them vote for which label
# if there's no consensus across votes, they'll pick randomly from the 3 options

def train_model(model,
                mini_train_data, 
                mini_train_labels):
    return model.fit(mini_train_data, mini_train_labels)

def predict_on_model(model,
                     dev_data, 
                     dev_labels):
    return model.predict(dev_data)

model_dict = {"bernoulli_alpha": BernoulliNB(alpha=.1),
             "bernoulli": BernoulliNB(),
             "gaussian_smoothed": GaussianNB(var_smoothing=.08),
             "knn_1": KNeighborsClassifier(1),
             "knn_3": KNeighborsClassifier(3),
             "knn_5": KNeighborsClassifier(5),
             "knn_5_distance": KNeighborsClassifier(5, weights="distance"),
             "knn_7_distance": KNeighborsClassifier(7, weights="distance")}


model_predictions_dict = {}

# loop over models to train them and predict on them
# add their predictions to the model predictions dict
# we use this dictionary so that we can train each model just once,
# even if we want to use that model in several ensembles
for model_name, model_function in model_dict.items():
    trained_model = train_model(model_function, mini_train_data, mini_train_labels)
    model_predictions = predict_on_model(trained_model, dev_data, dev_labels)
    model_predictions_dict[model_name] = model_predictions

In [ ]:
models_to_include = [ ["bernoulli_alpha", 
                     "gaussian_smoothed", 
                     "knn_1"],
                    ["bernoulli_alpha", 
                     "gaussian_smoothed", 
                     "knn_1",
                     "knn_3",
                     "knn_5"],
                    ["bernoulli_alpha", 
                     "gaussian_smoothed", 
                     "bernoulli",
                     "knn_1",
                     "knn_3",
                     "knn_5"],
                    ["bernoulli_alpha", 
                     "gaussian_smoothed", 
                     "knn_1",
                     "knn_3",
                     "knn_5_distance"],
                     ["bernoulli_alpha", 
                     "gaussian_smoothed", 
                     "knn_1",
                     "knn_1",
                     "knn_3",
                     "knn_5_distance"],
                     ["knn_1",
                     "knn_3",
                     "knn_5_distance"],
                    ["knn_1",
                     "knn_1",
                     "knn_3",
                     "knn_5_distance"],
                    ["bernoulli_alpha", 
                     "knn_1",
                     "knn_3",
                     "knn_5_distance"],
                     ["bernoulli_alpha", 
                     "knn_1",
                      "knn_1",
                     "knn_3",
                     "knn_5_distance"]]

# now, for a given model ensemble
# gather all predictions
for model_combo in models_to_include:
    predictions_for_ensemble = []
                     
    for model_name in model_combo:
        predictions_for_ensemble.append(model_predictions_dict[model_name])
    # get all the predictions 
                     
    # execute model voting,
    # determine ensemble accuracy
    accuracy_count = []
    for i in range(len(dev_data)):
        preds_for_i = [preds[i] for preds in predictions_for_ensemble]
        # this shuffle is how we ensure that random prediction will get chosen
        # if there isn't consensus
        np.random.shuffle(preds_for_i)
        counts = np.bincount(preds_for_i)
        voted_prediction = np.argmax(counts)
        if voted_prediction == dev_labels[i]:
            accuracy_count.append(1)
        else:
            accuracy_count.append(0)

    accuracy_for_ensemble = np.sum(accuracy_count) / 1000
    print("Tried ensemble with these models: {}".format(model_combo))
    print("Ensemble Accuracy: {}\n".format(accuracy_for_ensemble))

### STUDENT END ###

#P12()

Instead of new features, I decided to create an ensemble of models and have them predict via a vote. I wanted them to all train on the same mini_train_data, and I didn't use the blurring. I tried a bunch of different combinations of Naive Bayes and k-nn models with different hyperparameters. A subset of them are shown here. My best ensemble is the last one shown here. The accuracy I got on the mini train data, .895, is higher than what I got for any model trained on this set, other than the ones I did blurring on. 

A couple things that I learned along the way:
- For k-nn 5, changing the weights from "uniform" to "distance" really made a difference in the overall accuracy, whereas it didn't make much of a difference for k-nn 3. This makes sense, because it seems likely that once you are considering 5 training examples for classification, there's a reasonable chance that a couple of them are significantly further away and shouldn't be counted as much.
- k-nn 7 made things worse whether I switched it to "distance" or not. I suspect that this is a function of the small training size, and would change if we trained on more data (because there would likely be more examples that were true fits for the given dev example that would end up being in the nearest 7). 
- Because the k-nn models did better than the Naive Bayes models in general, ensembles that were more skewed towards k-nn models (even if some of the k-nn models were duplicates) did better than those that had more Naive Bayes models. That said, the best combinations did have some representation from Naive Bayes, which suggests that maybe the Naive Bayes models, while worse overall, are catching some of the cases that the k-nns aren't as good at. 
- Overall, this ensemble isn't that much better than just using a k-nn 1 (.895 vs .888). So, depending on the situation, it likely wouldn't be worth the additional computational complexity. 
